In [1]:
import geopandas as gpd
import tqdm
import os
import rasterio
import rasterio.merge
import rasterio.warp
import pandas as pd
import datetime
import numpy as np

In [2]:
import sys
sys.path.append('..')

In [3]:
import resample_worldcereal_cropmasks

In [4]:
bounding_filepath = '../../data/shapefiles/France_Regions.gpkg'
aez_geojson_filepath = '../../data/worldcereal/WorldCereal_AEZ.geojson'
reference_geotiff = '../../data/outputs/ref_mod09.ndvi.global_0.05_degree.2019.001.c6.v1.tif'

worldcereal_folderpath = '../../data/worldcereal/'
aez_s2_grid_folderpath = '../../data/outputs/s2_grid_level/worldcereal/'
s2_grid_level_folderpath = '../../data/outputs/s2_grid_level/aggregated_worldcereal/'
merged_product_folderpath = '../../data/outputs/s2_grid_level/aggregated_worldcereal'
resampled_cropmasks_folderpath = '../../data/outputs/s2_grid_level/resampled_cropmasks/'
output_folderpath = '../../data/outputs/merged_WC_cropmask/'

In [5]:
# 49 m 12.2 s
merged_cropmask_catalogue_df = \
resample_worldcereal_cropmasks.resample_worldcereal_cropmasks(
    roi_geom_filepath = bounding_filepath,
    worldcereal_folderpath = worldcereal_folderpath,
    worldcereal_aez_filepath = aez_geojson_filepath,
    ref_tif_filepath = reference_geotiff,
    aez_s2_grid_folderpath = aez_s2_grid_folderpath,
    s2_grid_level_folderpath = s2_grid_level_folderpath,
    merged_product_folderpath = merged_product_folderpath,
    resampled_cropmasks_folderpath = resampled_cropmasks_folderpath,
    output_folderpath = output_folderpath,
    products_to_merge = ['springcereals', 'wintercereals'],
    merged_product_name = 'cereals',
    out_products = ['cereals', 'temporarycrops'],
    s2_grid_res = 4,
    overwrite = False,
)

Dividing each aez's cropmask for products=['springcereals', 'temporarycrops', 'wintercereals'] by s2 grids:


  0%|          | 0/11 [00:00<?, ?it/s]

wintercereals tc-wintercereals 43153


100%|██████████| 11/11 [00:00<00:00, 1312.21it/s]


wintercereals tc-wintercereals 43170


...

Resampling cropmasks for products=['cereals', 'temporarycrops'] to target resolution


100%|██████████| 22/22 [00:00<00:00, 25546.70it/s]


Merging ['cereals', 'temporarycrops'] to bounds:


100%|██████████| 2/2 [00:00<00:00, 43.45it/s]


Co-register merged tifs with cropped ref_tif_filepath:


In [6]:
merged_cropmask_catalogue_df

,product,tif_filepath
0,cereals,../../data/outputs/merged_WC_cropmask/cereals.tif
1,temporarycrops,../../data/outputs/merged_WC_cropmask/temporar...


In [7]:
tif_filepaths = dict(zip(
    merged_cropmask_catalogue_df['product'],
    merged_cropmask_catalogue_df['tif_filepath'],
))

tif_filepaths

{'cereals': '../../data/outputs/merged_WC_cropmask/cereals.tif',
 'temporarycrops': '../../data/outputs/merged_WC_cropmask/temporarycrops.tif'}

In [8]:
masks_folderpath = '../../data/outputs/masks'
os.makedirs(masks_folderpath)

In [9]:
CEREAL_MASK_THRESHOLD = 10 # greater than and equal to
INTERP_MASK_THRESHOLD = 0  # greater than

In [10]:
def create_binary_mask(
    tif_filepath:str,
    out_mask_filepath:str,
    min_threshold,
    how='gte',
):
    with rasterio.open(tif_filepath) as src:
        ndarray = src.read()
        meta = src.meta.copy()
    
    out_ndarray = np.zeros(shape=ndarray.shape)
    if how == 'gte':
        out_ndarray[ndarray >= min_threshold] = 1
    elif how == 'gt':
        out_ndarray[ndarray > min_threshold] = 1
    else:
        raise ValueError(f"how must be either 'gte' or 'gt'.")

    _folderpath = os.path.split(out_mask_filepath)[0]
    os.makedirs(_folderpath, exist_ok=True)

    meta['dtype'] = rasterio.uint8
    meta['nodata'] = 0

    with rasterio.open(out_mask_filepath, 'w', **meta) as dst:
        dst.write(out_ndarray)

In [11]:
create_binary_mask(
    tif_filepath=tif_filepaths['cereals'], 
    out_mask_filepath=os.path.join(masks_folderpath, 'cereals.tif'),
    min_threshold=CEREAL_MASK_THRESHOLD,
    how='gte',
)

In [12]:
create_binary_mask(
    tif_filepath=tif_filepaths['temporarycrops'], 
    out_mask_filepath=os.path.join(masks_folderpath, 'temporarycrops.tif'),
    min_threshold=INTERP_MASK_THRESHOLD,
    how='gt',
)